In [18]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load your game dataset
df = pd.read_csv("Fake_Game_Dataset.csv")

X = df[['alpha', 'beta', 'gamma']].values
y = df['X'].values

# Standardize input features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split into training/test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [20]:
tfd = tfp.distributions
tfpl = tfp.layers

# Helper function for creating prior/posterior
def posterior_mean_field(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    c = np.log(np.expm1(1.))
    return tf.keras.Sequential([
        tfpl.VariableLayer(2 * n, dtype=dtype),
        tfpl.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=t[..., :n],
                       scale=1e-5 + tf.nn.softplus(c + t[..., n:])),
            reinterpreted_batch_ndims=1)),
    ])

def prior_trainable(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return tf.keras.Sequential([
        tfpl.VariableLayer(n, dtype=dtype),
        tfpl.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=t, scale=1),
            reinterpreted_batch_ndims=1)),
    ])


In [33]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Aliases
tfd = tfp.distributions
tfpl = tfp.layers

# Load data
df = pd.read_csv("Fake_Game_Dataset.csv")
X = df[['alpha', 'beta', 'gamma']].values
y = df['X'].values

# Standardize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

def prior_fn(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(n,)),
        tfpl.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=tf.zeros_like(t), scale=1.0),
            reinterpreted_batch_ndims=1))
    ])

def posterior_fn(kernel_size, bias_size=0, dtype=None):
    n = kernel_size + bias_size
    return tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(n * 2,)),
        tf.keras.layers.Dense(n * 2),  # learnable parameters
        tfpl.DistributionLambda(lambda t: tfd.Independent(
            tfd.Normal(loc=t[..., :n], scale=1e-5 + tf.nn.softplus(t[..., n:])),
            reinterpreted_batch_ndims=1))
    ])

inputs = tf.keras.Input(shape=(3,))
x = tfpl.DenseVariational(
    units=16,
    make_prior_fn=prior_fn,
    make_posterior_fn=posterior_fn,
    kl_weight=1.0 / X_train.shape[0],
    activation="relu"
)(inputs)

outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=50, batch_size=16, validation_split=0.1)

AttributeError: 'tuple' object has no attribute 'rank'

In [32]:
inputs = tf.keras.Input(shape=(3,))
x = tfpl.DenseVariational(
    units=16,
    make_prior_fn=prior_fn,
    make_posterior_fn=posterior_fn,
    kl_weight=1.0 / X_train.shape[0],
    activation="relu"
)(inputs)

AttributeError: 'tuple' object has no attribute 'rank'

In [30]:
inputs

<KerasTensor shape=(None, 3), dtype=float32, sparse=False, ragged=False, name=keras_tensor_5>